**Homework 2**

In this exercise we will retrieve some data from the NOAA webservice and do some data processing on it.

I have included the helper function from the book Notebooks, Chapter 3, 2 - using the weather api

***If you have serious trouble using/acccessing the API do not spend hours figuring it out let me know and worst case I can get you a csv file**

In [ ]:

import requests
import pandas as pd

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        A response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'RDBwieNtlQtPFszhHSvIukqJnIJVldda'
        },
        params=payload
    )


def get_item(name, what, endpoint, start=1, end=None):
    """
    Grab the JSON payload for a given field by name using binary search.

    Parameters:
        - name: The item to look for.
        - what: Dictionary specifying what the item in `name` is.
        - endpoint: Where to look for the item.
        - start: The position to start at. We don't need to touch this, but the
                 function will manipulate this with recursion.
        - end: The last position of the items. Used to find the midpoint, but
               like `start` this is not something we need to worry about.

    Returns:
        Dictionary of the information for the item if found otherwise 
        an empty dictionary.
    """
    # find the midpoint which we use to cut the data in half each time
    mid = (start + (end or 1)) // 2
    
    # lowercase the name so this is not case-sensitive
    name = name.lower()
    
    # define the payload we will send with each request
    payload = {
        'datasetid': 'GHCND',
        'sortfield': 'name',
        'offset': mid, # we will change the offset each time
        'limit': 1 # we only want one value back
    }
    
    # make our request adding any additional filter parameters from `what`
    response = make_request(endpoint, {**payload, **what})
    
    if response.ok:
        payload = response.json()

        # if response is ok, grab the end index from the response metadata the first time through
        end = end or payload['metadata']['resultset']['count']
        
        # grab the lowercase version of the current name
        current_name = payload['results'][0]['name'].lower()
        
        # if what we are searching for is in the current name, we have found our item
        if name in current_name:
            return payload['results'][0] # return the found item
        else:
            if start >= end: 
                # if our start index is greater than or equal to our end, we couldn't find it
                return {}
            elif name < current_name:
                # our name comes before the current name in the alphabet, so we search further to the left
                return get_item(name, what, endpoint, start, mid - 1)
            elif name > current_name:
                # our name comes after the current name in the alphabet, so we search further to the right
                return get_item(name, what, endpoint, mid + 1, end)    
    else:
        # response wasn't ok, use code to determine why
        print(f'Response not OK, status: {response.status_code}')

**CODE**
Here is the function from the book to retrieve data. Modify to retrieve the dataset for the two locations in the UAE:

* datasetid - GHCND
* stationid - GHCND:AEM00041194 & GHCND:AE000041196  (look at the datatypeid field for how to include multiple)
* locationid - CITY:AE000003
* For January of 2023

In [ ]:
response = make_request(
    'data', 
    {
        'datasetid': ,
        'stationid': ,
        'locationid': ,
        'startdate': ,
        'enddate': ,
        'datatypeid': ['TAVG', 'TMAX', 'TMIN'], # average, max, and min temperature
        'units': 'metric',
        'limit': 1000
    }
)
response.status_code

**CODE**
Read the 'results' of the web service call into a Pandas dataframe.

**CODE**

Use the "info" dataframe function to check the type of the columns

**CODE**

Data Type Conversion - change the column data types for:

* date - datetime
* datatype - categorical
* station - categorical

**CODE**

Use the describe function to summarize data on the CATEGORICAL columns only
Get a count for each unique value of datatype

**CODE**
 
Rename the value (temperature) column to temp_C

Create a new column called temp_F that calculated Farenheit on the formula (df.temp_C * 9/5) + 32

**CODE**

Let's analyze the data some.

Get the 10 hottest (TMAX) days, ordered by TMAX desc and Date asc

**CODE**

Get the 10 coldest (TMIN) days, ordered by TMIN asc and Date asc

**CODE**

* Create a new dataframe, indexed_df, that includes ONLY the TMAX data for station GHCND:AEM00041194

* Make the date column the index for this new dataframe

**CODE**

We haven't done much graphing yets but use the Padas Dataframe built in function "plot" to plot the temp_C over time. Might not work if you dont have the date column as the index.

This is/can be as simple as a single line of code, dont go crazy

**CODE**

Create another dataframe from the original DF that contains only TMIN for the station GHCND:AEM00041194

Set the date column to be the index

**CODE**

Plot the TMIN and TMAX dataframes together. This is as simple as calling the plot function on each dataframe and it should appear on the same graph